In [ ]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
main_dir = "D:\\Competitions\\DataStorm-Final-2021"
data_dir = os.path.join(main_dir,"data\\Data")
logs_dir = os.path.join(main_dir,"logs")
scripts_dir = os.path.join(main_dir,"src\\scripts")
model_dir = os.path.join(main_dir,"src\\models")

In [ ]:
filename = "datastorm_policy_data.csv"
dataframe = pd.read_csv(os.path.join(data_dir,filename))
sample = dataframe.sample(n=10)

In [ ]:
allColumns = list(dataframe.columns)
allColumns

snapshots = dataframe['policy_snapshot_as_on'].unique()

In [ ]:
categorical_columns = []
ordinal_columns = []
continuous_columns = []
remove_columns = ['child1_gender','child2_gender','child3_gender','child4_gender','child5_gender','main_holder_occupation','run_date','product_name']
target_column = 'Survived'

In [ ]:
dataframe.shape

In [ ]:
for each in allColumns:
    print(each,"   ",dataframe[each].unique().size)


In [ ]:
dataframe.groupby(by = ['policy_code','client_code']).count()

In [ ]:
dataframe.loc[(dataframe['policy_code'] == "POL0001")&(dataframe['client_code'] == "C105157")].sort_values(by=["policy_snapshot_as_on"],ascending=True)["policy_snapshot_as_on"]

In [ ]:
def add_index(df):
    df['policy_snapshot'] = pd.to_datetime(df['policy_snapshot_as_on'],format='%Y%m%d')
    df['unique'] = df['policy_snapshot_as_on'].apply(str)+" "+df['client_code']
    df.sort_values(by=["unique"],ascending=True,inplace=True)
    return df


def count_riders(df):
    rider_df = df[['rider1_prem','rider1_sum_assuared','rider2_prem','rider2_sum_assuared','rider3_prem','rider3_sum_assuared','rider4_prem','rider4_sum_assuared','rider5_prem','rider5_sum_assuared','rider6_prem','rider6_sum_assuared','rider7_prem','rider7_sum_assuared','rider8_prem','rider8_sum_assuared','rider9_prem','rider9_sum_assuared','rider10_prem','rider10_sum_assuared']]

    return rider_df

count_riders(sample)





In [ ]:
#add_index(sample)[['unique',"policy_snapshot_as_on"]]

In [ ]:
#add_index(dataframe)
#dataframe.to_csv(os.path.join(data_dir,"indexed.csv"))

In [ ]:
dataframe.isnull().sum()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
dataframe[target_column].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title(target_column)
ax[0].set_ylabel(target_column)
sns.countplot(target_column,data=dataframe,ax=ax[1])
ax[1].set_title(target_column)
plt.show()

In [ ]:
def categorical_bar(column):
    f,ax=plt.subplots(1,2,figsize=(18,8))
    dataframe[[column,target_column]].groupby([column]).mean().plot.bar(ax=ax[0])
    ax[0].set_title(f'{target_column} vs {column}')
    sns.countplot(column,hue=target_column,data=dataframe,ax=ax[1])
    ax[1].set_title(f'{column}:{target_column} vs other')
    plt.show()

In [ ]:
categorical_bar('Sex')

In [ ]:
categorical_bar('Pclass')